### 기존 데이터셋 확인

In [ ]:
from datasets import load_dataset
tesat = load_dataset("overfit-brothers/TESAT",split="train")
financial_mmlu = load_dataset("overfit-brothers/financial-mmlu-ko-cot-revised",split="train")


In [145]:
import re
def process_blank(example):
    example = example.replace('\n', ' ')
    example = ' '.join(example.split())
    return example


def parse_choices(text):
    # 정규표현식 패턴: 알파벳+점+공백 다음에 오는 텍스트를 캡처
    pattern = r'([A-E])\.\s+([^A-E]+?)(?=[A-E]\.|$)'
    
    # findall로 모든 매칭을 찾아서 딕셔너리로 변환
    choices = dict(re.findall(pattern, text))
    
    # 각 선택지의 앞뒤 공백 제거
    choices = {k: v.strip() for k, v in choices.items()}
    
    return choices

In [ ]:
tesat[0]

In [129]:
tesat_list = []
for datum in tesat:
    question = process_blank(datum['question']).replace('### 질문: ','')
    answer_choice = datum['answer'].split("\n\n정답 ")[-1].rstrip('.')
    answer = process_blank(datum['answer'].replace("\n\n정답 ","### 정답: ")).rstrip('.')

    original_choices = parse_choices(question)
    res_dict = {
        'original_question': question,
        'answer': answer,
        'answer_choice': answer_choice,
    }
    for k,v in original_choices.items():
        res_dict[k] = v
    tesat_list.append(res_dict)

In [ ]:
tesat_list

In [146]:
HJ_list = []
for datum in hand:
    question = process_blank(datum['question']).replace('### 질문: ','')
    answer_choice = datum['answer'].split("\n\n정답 ")[-1].rstrip('.')
    answer = process_blank(datum['answer'].replace("\n\n정답 ","### 정답: ")).rstrip('.')

    original_choices = parse_choices(question)
    res_dict = {
        'original_question': question,
        'answer': answer,
        'answer_choice': answer_choice,
    }
    for k,v in original_choices.items():
        res_dict[k] = v
    HJ_list.append(res_dict)

In [13]:
financial_mmlu_list = []
for datum in financial_mmlu:
    question = datum['question']
    answer = datum['answer']
    answer_counts = {'A': answer.count('A'), 'B': answer.count('B'), 'C': answer.count('C'),
                'D': answer.count('D'), 'E': answer.count('E'), 'F': answer.count('F')}
    answer_choice = max(answer_counts, key=answer_counts.get)
    original_choices = parse_choices(question)
    res_dict = {
    'original_question': question,
    'answer': answer,
    'answer_choice': answer_choice,
}
    for k,v in original_choices.items():
        res_dict[k] = v
    financial_mmlu_list.append(res_dict)

In [14]:
financial_mmlu_list[0]

{'original_question': '금리의 종류에 대한 설명으로 바르지 않은 것은? ### 선택지: A. 변동금리는 시장금리 변동에 따른 위험을 자금공급자가 부담하게 된다 B. 피셔방정식에 의하면 실질금리는 명목금리에서 기대인플레이션을 차감하면 구할 수 있다. C. 복리는 원금에 대한 이자뿐 아니라 이자에 대한 이자도 함께 계산하는 방법이 다. D. 실효금리는 이자지급방법, 상환방법, 수수료, 세금 등을 감안한 후 차입자가 실질적으로 부담하는 순자금조달비용을 말한다. E. 채권시장에서는 금리보다 수익률이라는 용어를 더 많이 사용한다.',
 'answer': 'A 문항은 변동금리에 대한 설명으로, 변동금리는 시장금리 변동에 따라 차입자가 금리 변동의 위험을 부담하는 것이 일반적입니다. 따라서 자금공급자가 위험을 부담한다는 설명은 바르지 않습니다. B 문항은 피셔방정식에 대한 설명으로, 실질금리는 명목금리에서 기대인플레이션을 차감하여 구할 수 있다는 설명이 맞습니다. C 문항은 복리에 대한 설명으로, 복리는 원금뿐만 아니라 이자에 대한 이자도 계산하는 방법이므로 맞는 설명입니다. D 문항은 실효금리에 대한 설명으로, 실효금리는 차입자가 실질적으로 부담하는 순자금조달비용을 의미하므로 맞는 설명입니다. E 문항은 채권시장에서 금리보다 수익률이라는 용어가 더 많이 사용된다는 설명으로, 일반적으로 맞는 설명입니다. ### 정답: A',
 'answer_choice': 'A',
 'A': '변동금리는 시장금리 변동에 따른 위험을 자금공급자가 부담하게 된다',
 'B': '피셔방정식에 의하면 실질금리는 명목금리에서 기대인플레이션을 차감하면 구할 수 있다.',
 'C': '복리는 원금에 대한 이자뿐 아니라 이자에 대한 이자도 함께 계산하는 방법이 다.',
 'D': '실효금리는 이자지급방법, 상환방법, 수수료, 세금 등을 감안한 후 차입자가 실질적으로 부담하는 순자금조달비용을 말한다.',
 'E': '채권시장에서는 금리보다 수익률이라는 용어를 더 많이 사용한다.'}

선택지 순서 셔플

In [18]:
tesat_list[0]

{'original_question': '다음 중 시장실패라고 볼 수 없는 상황은? ### 선택지: A. 규모의 경제가 무한정 작용하고 있다. B. 독점기업이 존재한다 C. 기업이 적자가 나는데도 불구하고 구제금융을 받아 연명하고 있다. D. 공장이 많이 들어서자 주위의 강이 오염되어 버렸다. E. 어떤 중소기업이 아주 수익성이 좋은 투자계획안을 가지고 있음에도 불구하고 단지 중소기업이라는 이유 때문에 대출을 못 받고 있다.',
 'answer': '시장의 실패는 경제학의 기본 명제인 자원의 최적 배분을 해결하지 못하는 시장 상황을 말한다. 가격 기능이 작동하지 않아 생산량이 사회 최적 수준보다 많아지거나 적어질 경우 자원의 최적 배분이 이뤄지지 않아 시장은 실패한다. 시장 실패는 규모의 경제가 작용하거나 기업의 기술개발 등으로 한두 개 기업이 시장을 독과점하는 경우, 환경 오염 비용이 사회에 전가되는 등 외부 효과가 발생하는 경우, 국방 치안처럼 재화나 서비스의 공급을 시장에 맡기기 힘든 경우(공공재) 등을 들 수 있다. 시장이 실패하면 보통 정부가 개입하지만 정부 역시 능력의 한계 불완전한 시장 정보 등으로 임시방편적인 처방을 내놓거나 정경유착 등의 부작용을 일으킬 수 있다. 이를 정부실패라고 한다. 문제의 보기 3번은 이윤을 창출하지 못하는 적자 기업이 시장에서 퇴출되지 않고 정부의 구제 금융에 의존하도록 하는 것은 시장 실패가 아닌 정부 실패다. 구제책은 시장의 정상적인 작동에 오히려 장애가 되고 불황을 장기화시킬 수 있다.### 정답: C',
 'answer_choice': 'C',
 'A': '규모의 경제가 무한정 작용하고 있다.',
 'B': '독점기업이 존재한다',
 'C': '기업이 적자가 나는데도 불구하고 구제금융을 받아 연명하고 있다.',
 'D': '공장이 많이 들어서자 주위의 강이 오염되어 버렸다.',
 'E': '어떤 중소기업이 아주 수익성이 좋은 투자계획안을 가지고 있음에도 불구하고 단지 중소기업이라는 이유 때문에 대출을 못 받고 있

In [149]:
from datasets import DatasetDict, Dataset
import random
from copy import deepcopy
from tqdm.notebook import tqdm
def shuffle_dataset(data_list, num_augmentations):
    # 새로운 DatasetDict 생성
    augmented_examples = []
    # 각 예제에 대해 증강 수행
    for example in tqdm(data_list):
            
        # 추가 증강 데이터 생성
        for _ in range(num_augmentations):
            PARSE_POSSIBLE = True
            new_example = deepcopy(example)
            
            # 현재 정답 찾기
            original_question = example['original_question'].split("### 선택지:")[0].strip()
            original_answer = example['answer_choice']
            choices = []
            choice_contents = {}
            
            # 가능한 모든 선택지 키 검사
            possible_choices = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
            for choice in possible_choices:
                if choice in example:
                    choices.append(choice)
                    choice_contents[choice] = example[choice]
            
            # answer_choice가 choices 안에 있는지 확인
            if original_answer not in choices:
                continue
                
            # 선택지 순서 섞기 (정답 위치가 바뀔 때까지)
            max_attempts = 0  # 최대 시도 횟수 설정
            while True:
                new_choices = choices.copy()
                random.shuffle(new_choices)
                max_attempts += 1
                if new_choices.index(original_answer) != choices.index(original_answer):
                    break
                if max_attempts >= 100:  # 최대 시도 횟수 초과 시 중단
                    PARSE_POSSIBLE = False
                    break
            if not PARSE_POSSIBLE:
                continue
            # 새로운 순서로 선택지 재배치
            choice_text_list = []
            for i, new_choice in enumerate(new_choices):
                new_example[choices[i]] = choice_contents[new_choice]
                if new_choice == original_answer:
                    new_example['answer'] = choices[i]
                choice_text_list.append(f"{choices[i]}.{choice_contents[new_choice]}")
                
            new_example['question'] = f'{original_question} ### 선택지: {" ".join(choice_text_list)}'
            
            choices_content = []
            for choice in choices:
                choices_content.append(new_example[choice])
                del new_example[choice]
                
            new_example['choices'] = choices_content
            del new_example['answer_choice']
            del new_example['original_question']
            augmented_examples.append(new_example)    
    
    return augmented_examples

In [45]:
tesat_list[124]

{'original_question': '다음은 주식의 가치를 분석하기 위한 지표들에 대한 설명이다. 틀린 것은? ### 선택지: A. 주당순이익(EPS)은 법인세 공제 후 순이익을 주식 수로 나눈 값이다. B. 주가수익비율(PER)은 수익성 측면에서 주가를 판단하는 지표다. C. 주식 투자의 판단지표로 배당수익률은 고려 대상이 아니다. D. 주가순자산비율(PBR)은 주가를 1주당 순자산가액으로 나눈 값이다. E. 주가수익비율(PER)은 주가를 법인세 공제 후 1주당 순이익으로 나눈 것이다.',
 'answer': '현재 주가가 고평가돼 있는지，저평가돼 있는지 또는 이 주식을 살지,말지를 판단하는 데는 다양한 지표들이 사용된다. 주당순이익(EPS)은 당기순익을 주식 수로 나눈 값이다. 두 기업의 당기순이익이 같아도 주식 수가 더 많은 기업,즉 EPS가 낮은 기업의 주가가 낮은 게 일반적이다. 주가수익비율(PER)은 주가를 주당순이익으로 나눈 수치로 순이익에 비해 주가가 얼마나 높은지 또는 낮은지를 나타내는 지표다. PER가 낮을수록 저평가돼 있어 성장 가능성이 높고，PER가 높을수록 고평가돼 있어 성장 가능성이 낮다. 하지만 PER가 낮아도 부실한 기업보다는 PER가 높더라도 우량한 대기업이 더 매력적인 구매력을 갖고 안정성이 높다. 즉 PER는 수익성 측면에서 주가를 판단하는 지표이며 안정성이 포함돼 있지 않으므로 투자에 절대적인 수치가 아니다. 주가순자산비율(PBR)은 주가를 주당 순자산으로 나눈 것이다. 순자산이란 대차대조표의 자산에서 부채를 차감한 후의 자산을 말한다. PBR는 재무내용면에서 주가를 판단하는 지표다. 어떤 기업의 PBR가 낮다면 부동산이나 설비시설，기타 유동자금들을 많이 보유하고 있는데 주가는 낮은 알짜배기 기업이라고 볼 수 있다. 배당수익률은 현 주가나 매입 주가에 대한 전년도 배당금 비율이다. 배당금이 많다면 주가가 많이 오르지 않아도 어느 정도의 안전마진을 확보할 수 있다. 예를 들어 매수가 대비 시가배당수익률이 금리 이상이라면 주

In [ ]:
# list_to_augment = [tesat_list,financial_market_list,financial_accounting_list,financial_mmlu_list,tat_fat_list]

tesat_shuffled = shuffle_dataset(tesat_list,2)
print('tesat_shuffled',len(tesat_shuffled))
financial_mmlu_shuffled = shuffle_dataset(financial_mmlu_list,2)
print('financial_mmlu_shuffled',len(financial_mmlu_shuffled))


In [ ]:
import pandas as pd
from datasets import load_dataset, Dataset
import random
from litellm import completion, batch_completion
import os
import litellm
os.environ["OPENAI_API_KEY"] = "sk-XXX"


In [155]:
def make_cot_prompt(question):
    prompt = f"""
    문제를 읽고 파악한 후에, 한국어로 풀이과정과 정답을 알려주세요. 
    
    - 요구사항: 
    1. 문제를 읽고 파악하고 문제에서 요구하는 목표를 간략하게 1문장으로 말하세요.
    2. 풀이과정은 논리적 사고 흐름에 따라 생각의 사슬 형식으로 설명하세요. 
    3. 도출된 답안을 제시하고 마지막에 반드시 다음과 같이 정답의 기호(알파벳 대문자)를 한번 더 언급해 주세요. ### 정답: [정답 기호]
    
    답변 형식:
    1) 문제 분석: [문제의 핵심 내용 정리]
    2) 풀이 과정: [단계별 설명,문장 형식]
    3) 최종 정답: ### 정답: [정답 기호]
    <중요사항>
    문제의 정답은 무조건 1개 존재하며, 복수 정답의 존재나 정답 불가능 등의 경우는 존재하지 않습니다.
    </중요사항>
    
    문제: 
    {question}
    """
    return prompt

In [156]:
def make_answer_litellm(example:list):
    qrys = []
    for datum in example:
        question = datum['question']
        messages = [
        {"content":"You are Korean expert in finance and economics, and accounting.","role":"system"},
        { "content": make_cot_prompt(question),"role": "user"}]
        qrys.append(messages)
    responses = batch_completion(
    model="gpt-4o-2024-08-06",
    messages = qrys
)   
    resps = [i.choices[0].message.content for i in responses]
    total_len = len(example)
    cnt = 0
    res_list = []
    for datum, response in zip(example, resps):
        label_answer = datum['answer']
        hypothesis_answer = response.split("### 정답: ")[-1]
        if hypothesis_answer not in ['A', 'B', 'C', 'D', 'E', 'F', 'G'] or hypothesis_answer != label_answer:
            continue
        res_list.append({'question': datum['question'], 'answer': process_blank(response)})
        cnt += 1
    print(f"augmented {cnt}/{total_len}")
    return res_list

In [ ]:
tesat_shuffled_with_answer = make_answer_litellm(tesat_shuffled)

financial_mmlu_shuffled_with_answer = make_answer_litellm(financial_mmlu_shuffled)

- shuffled tesat 정답률 77%
- shuffled financial mmlu 정답률 83%

In [131]:
from datasets import Dataset
tesat_list_filtered = [{
    'question': item['original_question'],
    'answer': item['answer']
} for item in tesat_list]
tesat_final = Dataset.from_list(tesat_list_filtered+tesat_shuffled_with_answer)

In [ ]:
tesat_final

In [ ]:

financial_mmlu_list_filtered = [{
    'question': item['original_question'],
    'answer': item['answer']
} for item in financial_mmlu_list]
financial_mmlu_final = Dataset.from_list(financial_mmlu_list_filtered + financial_mmlu_shuffled_with_answer)

In [ ]:
tesat_final.push_to_hub('overfit-brothers/TESAT_augmented',private=True)
